In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Paths
DATASET_PATH = r"C:\Users\Pc\Desktop\sign\sign language ti speech translater\image"  # dataset folder containing subfolders for each label
IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 25


In [3]:
# Data generators
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 788 images belonging to 26 classes.
Found 185 images belonging to 26 classes.


In [4]:
# CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # number of classes auto from generator
])

c:\Users\Pc\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Compile
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
# Train
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20
)


Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.9829 - loss: 0.0450 - val_accuracy: 1.0000 - val_loss: 0.0023
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.9762 - loss: 0.1188 - val_accuracy: 1.0000 - val_loss: 0.0097
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 0.9867 - loss: 0.0447 - val_accuracy: 0.9946 - val_loss: 0.0230
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.9941 - loss: 0.0367 - val_accuracy: 0.9946 - val_loss: 0.0085
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 0.9941 - loss: 0.0273 - val_accuracy: 0.9946 - val_loss: 0.0056
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.9881 - loss: 0.0351 - val_accuracy: 1.0000 - val_loss: 0.0027
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 88ms/step - accuracy: 0.9970 - loss: 0.0166 - val_accuracy: 0.9946 - val_loss: 0.0117
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.9900 - loss: 0.0260 - val_accuracy: 1.0000 - v

In [8]:
# Save model
model.save("sign_language_model.h5")

In [9]:
# Print accuracy
print("Final Training Accuracy:", history.history['accuracy'][-1])
print("Final Validation Accuracy:", history.history['val_accuracy'][-1])

Final Training Accuracy: 0.9949238300323486
Final Validation Accuracy: 0.9945945739746094


In [10]:
# Define class labels for A-Z
class_labels = [chr(i) for i in range(ord('A'), ord('Z') + 1)]


In [11]:
import cv2
import numpy as np

# Path to test image
img_path = r"image/E/1.png"

In [12]:
# Load and preprocess the image
img = cv2.imread(img_path)
img_resized = cv2.resize(img, (64, 64))  # Use the size your CNN expects
img_normalized = img_resized / 255.0
img_input = np.expand_dims(img_normalized, axis=0)

In [13]:
# Predict the sign
pred = model.predict(img_input)
class_index = np.argmax(pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


In [14]:
predicted_letter = class_labels[class_index]
print("Predicted Sign:", predicted_letter)

Predicted Sign: E
